In [13]:
import rasterio
import numpy as np
from rasterio.plot import adjust_band
import matplotlib.pyplot as plt
from rasterio.plot import reshape_as_raster, reshape_as_image
from rasterio.plot import show
from itertools import product
from rasterio.windows import Window
from pyproj import Proj, transform
import random
import math

In [14]:
label_dataset = rasterio.open("/deep_data/data/land_cover_data/landcover_reproject.tif")
label_image = label_dataset.read()

tiles = ["028012", "028011"]

landsat_dataset = list(i for i in range(len(tiles)))
count = 0
for tile in tiles:
    landsat_dataset[count] = rasterio.open("/deep_data/data/combined/combined" + tile + ".tif")
    count+=1

In [4]:
# What is the raster's projection?
image_proj = landsat_dataset[1].crs # 4326
print('Image projection:')
print(image_proj)

# What is the raster's projection?
label_proj = label_dataset.crs
print('Labels projection:')
print(label_proj)

Image projection:
PROJCS["Albers",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378140,298.2569999999957,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]
Labels projection:
EPSG:32618


In [15]:
def make_pixel_locations(image_datasets, train_count, val_count, tile_size):
    ### this function pulls out a randomly selected number of pixels from a list of raster datasets
    ### and returns a list of training pixel locations and image indices 
    ### and a list of validation pixel locations and indices
    
    ## future improvements could make this select classes evenly
    train_pixels = []
    val_pixels = []
    
    buffer = math.ceil(tile_size/2)
    
    total_count = train_count + val_count
    for index, image_dataset in enumerate(image_datasets):
        #randomly pick `count` num of pixels from each dataset
        img_height, img_width = image_dataset.shape
        
        rows = range(0+buffer, img_height-buffer)
        columns = range(0+buffer, img_width-buffer)
        #rows_sub, columns_sub = zip(*random.sample(list(zip(rows, columns)), total_count))
        
        points = random.sample(set(product(rows, columns)), total_count)
        
        dataset_index = [index] * total_count
        dataset_pixels = list(zip(points, dataset_index))
        
        train_pixels += dataset_pixels[:train_count]
        val_pixels += dataset_pixels[train_count:]
        
        
    return (train_pixels, val_pixels)


def tile_generator(image_datasets, label_dataset, tile_height, tile_width, pixel_locations, batch_size):
    ### this is a keras compatible data generator which generates data and labels on the fly 
    ### from a set of pixel locations, a list of image datasets, and a label dataset
    
    # pixel locations looks like [r, c, dataset_index]
    label_image = label_dataset.read()
    label_image[label_image == 255] = 1

    c = r = 0
    i = 0
    
    outProj = Proj(label_dataset.crs)

    # assuming all images have the same num of bands
    band_count = image_datasets[0].count
    class_count = len(np.unique(label_image))
    buffer = math.ceil(tile_height / 2)
  
    while True:
        image_batch = np.zeros((batch_size, tile_height, tile_width, band_count-1)) # take one off because we don't want the QA band
        label_batch = np.zeros((batch_size, class_count, tile_height, tile_width, 1))
        b = 0
        while b < batch_size:
            # if we're at the end  of the data just restart
            if i >= len(pixel_locations):
                i=0
            c, r = pixel_locations[i][0]
            dataset_index = pixel_locations[i][1]
            i += 1
            
            # GET GPS COORDINATES AND USE THESE TO CREATE LABEL TILE
            (x, y) = image_datasets[dataset_index].xy(r, c)
            inProj = Proj(image_datasets[dataset_index].crs)
            if inProj != outProj:
                x,y = transform(inProj,outProj,x,y)
            #    
            
        
                
            tile = image_datasets[dataset_index].read(list(np.arange(1, band_count+1)), window=Window(c-buffer, r-buffer, tile_width, tile_height))
            Labeltile = label_dataset.read(0, window=Window(x-buffer, y-buffer, tile_width, tile_height))

            if np.amax(tile) == 0: # don't include if it is part of the image with no pixels
                pass
            elif np.isnan(tile).any() == True or -9999 in tile: 
                # we don't want tiles containing nan or -999 this comes from edges
                # this also takes a while and is inefficient
                pass
            elif tile.shape != (band_count, tile_width, tile_height):
                print('wrong shape')
                print(tile.shape)
                # somehow we're randomly getting tiles without the correct dimensions
                pass
            elif np.isin(tile[7,:,:], [352, 368, 392, 416, 432, 480, 840, 864, 880, 904, 928, 944, 1352]).any() == True:
                # make sure pixel doesn't contain clouds
                # this is probably pretty inefficient but only checking width x height for each tile
                # read more here: https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/LSDS-1873_US_Landsat_ARD_DFCB_0.pdf
                #print('Found some cloud.')
                #print(tile[7,:,:])
                pass
            else:
                tile = adjust_band(tile[0:7])
                # reshape from raster format to image format
                reshaped_tile = reshape_as_image(tile)

                # find gps of that pixel within the image
                (x, y) = image_datasets[dataset_index].xy(r, c)

                # convert the point we're sampling from to the same projection as the label dataset if necessary
                inProj = Proj(image_datasets[dataset_index].crs)
                if inProj != outProj:
                    x,y = transform(inProj,outProj,x,y)

                # reference gps in label_image
                row, col = label_dataset.index(x,y)

                # find label
                label = label_image[:, row, col]
                # if this label is part of the unclassified area then ignore
                if label == 0 or np.isnan(label).any() == True:
                    pass
                else:
                    # add label to the batch in a one hot encoding style
                    label_batch[b][label] = 1
                    image_batch[b] = reshaped_tile
                    b += 1
        yield (image_batch, label_batch)

In [29]:
tile_size = 11
batch_size = 25
label_image[label_image == 255] = 1
epochs = 100
classNum = len(np.unique(label_image))
(train_pixels, val_pixels) = make_pixel_locations(landsat_dataset, 50000, 10000, tile_size)


In [30]:
test_pixels = val_pixels[:math.ceil(len(val_pixels)/3)]
val_pixels = val_pixels[math.ceil(len(val_pixels)/3):]

In [32]:
# input image dimensions
img_rows, img_cols = tile_size, tile_size
img_bands = landsat_dataset[0].count

input_shape = (img_rows, img_cols, img_bands)
print(input_shape)

print(len(train_pixels))
print(len(val_pixels))
print(len(test_pixels))




(11, 11, 7)
100000
13333
6667


In [19]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [33]:
model = Sequential()
model.add(Conv2D(100, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(classNum, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 9, 9, 100)         6400      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 4, 4, 100)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 2, 2, 128)         115328    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               66048     
_________________________________________________________________
dense_10 (Dense)             (None, 23)                11799     
Total para

In [36]:
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
metrics=['accuracy']
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)

In [37]:
model.fit_generator(generator=tile_generator(landsat_dataset, label_dataset, tile_size, tile_size, train_pixels, batch_size), 
                    steps_per_epoch=len(train_pixels) // batch_size, epochs=epochs, verbose=1,
                    validation_data=tile_generator(landsat_dataset, label_dataset, tile_size, tile_size, val_pixels, batch_size),
                    validation_steps=len(val_pixels) // batch_size)

Epoch 1/100
4000/4000 [==============================] - 100s 25ms/step - loss: 1.6693 - acc: 0.4712 - val_loss: 1.5457 - val_acc: 0.5140
Epoch 2/100
4000/4000 [==============================] - 93s 23ms/step - loss: 1.5901 - acc: 0.4897 - val_loss: 1.4939 - val_acc: 0.5220
Epoch 3/100
4000/4000 [==============================] - 88s 22ms/step - loss: 1.5217 - acc: 0.5117 - val_loss: 1.8562 - val_acc: 0.3622
Epoch 4/100
4000/4000 [==============================] - 89s 22ms/step - loss: 1.5029 - acc: 0.5163 - val_loss: 1.4635 - val_acc: 0.5355
Epoch 5/100
4000/4000 [==============================] - 93s 23ms/step - loss: 1.5064 - acc: 0.5114 - val_loss: 1.5018 - val_acc: 0.5287
Epoch 6/100
4000/4000 [==============================] - 91s 23ms/step - loss: 1.4810 - acc: 0.5184 - val_loss: 1.6410 - val_acc: 0.5051
Epoch 7/100
4000/4000 [==============================] - 84s 21ms/step - loss: 1.4253 - acc: 0.5387 - val_loss: 1.4868 - val_acc: 0.5294
Epoch 8/100
4000/4000 [=================

In [40]:
model.evaluate_generator(generator=tile_generator(landsat_dataset, label_dataset, tile_size, tile_size, test_pixels, batch_size), 
                        steps=len(test_pixels) // batch_size,
                         verbose=1)

266/266 [==============================] - 13s 51ms/step


[5.111856063505761, 0.3353383461279528]